# Introdução

**Nesse primeiro notebook, nosso propósito é fazer uma análise inicial dos dados do dataset de bicicletas compartilhadas de nova iorque e entender suas peculiaridades nos dados.**

Dados Completos da Fonte: https://s3.amazonaws.com/tripdata/index.html

Acumulamos os dados no arquivo "0. Download Dados"

## Próximos Passos

**Análise de Dados:**
* [x] Análise básica dos dados
* [x] Limpeza dos dados
* [ ] Visualização de Dados Geográficos
    * [ ] Visualização dos pontos de saída e chegada
* [ ] Calcular distância absoluta de cada estação a outra.
* [ ] Analisar as estações mais populares
    * [ ] Separado por dia de semana / fim de semana
* [ ] Analisar os horários de pico e distribuição dos horários geral e por estação
* [ ] Analisar distribuição por dia, mês, ano da semana geral e por estação
* [ ] Analise de sazonalidade
* [ ] Relação do tempo da viagem com a idade e gênero da pessoa
* [ ] Distribuição por estação de idade e gênero
* [ ] Análise de corridas em grupo - mesmo inicio e fim? - pensar como avaliar isso

**Modelos:**
* [ ] Predizer para uma estação, em um dia da semana e horário, quantas corridas terá

**Tempo Real:**
* [ ] Obter dados em tempo real pela API
* [ ] Aplicar o modelo de predição nos dados reais, predizer para próximas horas
* [ ] Criar um dashboard em tempo real

# Importações

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

In [ ]:
from haversine import haversine

In [ ]:
px.set_mapbox_access_token("pk.eyJ1Ijoicm1jbnJpYmVpcm8iLCJhIjoiY2s4MHh5b3ZiMGtsbTNkcGFuazR1dWc4diJ9._aDTNPlmw3Nt6QSMm3YgmQ")

**Importante**: Devido a restrição de espaço no Github, baixe os dados do Drive e coloque localmente na pasta 'Dados'.
https://drive.google.com/drive/folders/144_0BrEXS3Z1VslYxr1rP2Wr6bSm4tDf?usp=sharing

In [ ]:
df_trips = pd.read_parquet("../Dados/citibike2019_sample.parquet")

# Análise Inicial

**Entendendo os tipos de dados e suas distribuições e comportamentos.**

In [ ]:
df_trips.info()

**Verificando colunas com valores nulos.**

Precisaremos dropar essas linhas porque os IDs são relevantes na nossa análise.

In [ ]:
df_trips.isna().any()

**Mapeamento de Colunas para padronizar os nomes.**

In [ ]:
df_trips = df_trips.rename(
    columns={
        'starttime': "Start Time", 
        'stoptime': "Stop Time", 
        'start station id': "Start Station ID", 
        'start station name': "Start Station Name",
        'start station latitude': "Start Station Latitude", 
        'start station longitude': "Start Station Longitude", 
        'end station id': "End Station ID",
        'end station name': "End Station Name", 
        'end station latitude': "End Station Latitude", 
        'end station longitude': "End Station Longitude",
        'bikeid': "Bike ID", 
        'usertype': "User Type", 
        'birth year': "Birth Year", 
        'gender': "Gender",
        'tripduration': "Trip Duration"
    }
)

In [ ]:
df_trips.columns

In [ ]:
df_trips["Start Time"] = pd.to_datetime(df_trips["Start Time"])
df_trips["Stop Time"] = pd.to_datetime(df_trips["Stop Time"])

In [ ]:
# Adicionando Coluna de Tempo Total em Minutos
df_trips["Trip Duration Minutes"] = df_trips["Trip Duration"]/60.0

In [ ]:
df_trips.describe()

**Observações de Dados:**

* Máximo de Trip duration muito alta
* Longitude possui valores zerados, indicando trips não finalizadas. Provavelmente vamos dropar.
* Birth Year mínimo de 1857 - precisamos ver a veracidade dessa coluna
* Gender 0, 1, 2 - precisamos verificar os significados
* Trip Duration Minutes com máximo muito alto
* Dados nulos de ID e Name

## Tratando Dados Nulos

In [ ]:
df_trips = df_trips.dropna()

In [ ]:
df_trips.info()

In [ ]:
df_trips.loc[:, "Start Station ID"] = df_trips.loc[:, "Start Station ID"].astype(int).values
df_trips.loc[:, "End Station ID"] = df_trips.loc[:, "End Station ID"].astype(int).values

In [ ]:
df_trips.head()

## Análise de Trip Duration - Semelhante a Trip_Duration_in_min

In [ ]:
trip_durations = df_trips["Trip Duration Minutes"]

In [ ]:
trip_durations[trip_durations < 100].plot.hist(bins=30)

## Análise de Birth Year

In [ ]:
birth_years = df_trips["Birth Year"]

In [ ]:
birth_years.plot.hist(bins=30)

Criação de Nova Coluna de Idade:

In [ ]:
data_captura = 2019
df_trips["Age"] = data_captura - df_trips["Birth Year"]

In [ ]:
df_trips["Age"].plot.hist()

In [ ]:
df_trips["Age"].plot.box()

## Unicidade de (latitude, longitude) para cada uma das Stops

Será que os dados de latitude e longitude das stops é consistente ou aparecem múltiplos (lat, long) para um mesmo stop?

In [ ]:
# Obtendo as colunas relacionadas a start stops
start_stations = df_trips[['Start Station ID', 'Start Station Latitude', 'Start Station Longitude', 'Start Station Name']]
start_stations.columns = ['Station ID', 'Station Latitude', 'Station Longitude', 'Station Name']

In [ ]:
# Obtendo as colunas relacionadas a end stops
end_stations = df_trips[['End Station ID', 'End Station Latitude', 'End Station Longitude', 'End Station Name']]
end_stations.columns = ['Station ID', 'Station Latitude', 'Station Longitude', 'Station Name']

In [ ]:
# Uniao das stops, ja que ser de inicio ou fim nao importa
stations = pd.concat([start_stations, end_stations], axis=0, ignore_index=True)

In [ ]:
stations.head()

In [ ]:
# Verificando se todos os grupos de Station ID possuem apenas 1 linha após dropar duplicatas
(stations.drop_duplicates().groupby("Station ID").size() == 1).all()

In [ ]:
stations.drop_duplicates().groupby("Station ID").size().sort_values()

### Definindo um Dataframe mapeador das stations

Pode ser útil mais tarde termos um mapeador da station id para a lat, long

In [ ]:
df_stations = stations.drop_duplicates().set_index('Station ID')

In [ ]:
df_stations.head()

# Data Augmentation

In [ ]:
df_trips["isweekend"] = df_trips["Start Time"].dt.weekday >= 5

In [ ]:
df_trips["weekday"] = df_trips["Start Time"].dt.weekday

# Limpeza de Dados

**Observações de Dados:**

* Máximo de Trip duration muito alta
* Longitude possui valores zerados, indicando trips não finalizadas. Provavelmente vamos dropar.
* Birth Year mínimo de 1900 - precisamos ver a veracidade dessa coluna
* Gender 0, 1, 2 - precisamos verificar os significados
* Trip_Duration_in_min com máximo muito alto
* Nenhum dado nulo no dataset

In [ ]:
df_trips_filtrado = df_trips

## Trip Duration

In [ ]:
trip_duration = df_trips_filtrado["Trip Duration"]

In [ ]:
Q1 = trip_duration.quantile(0.25)
Q3 = trip_duration.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
duration_filter = ~((trip_duration < (Q1 - 1.5 * IQR)) |(trip_duration > (Q3 + 1.5 * IQR)))
df_trips_filtrado = df_trips[duration_filter]

In [ ]:
df_trips_filtrado

## Latitude e Longitude Zerados

A filtragem anterior resolveu o problema.

In [ ]:
df_trips_filtrado.describe()

# Análise das Distâncias 'Percorridas'

In [ ]:
df_trips["Distâncias"] = df_trips.apply(lambda linha:haversine((linha['Start Station Latitude'] , linha['Start Station Longitude']), (linha['End Station Latitude'], linha['End Station Longitude'])),axis=1)

# Análise das Estações Populares

In [ ]:
df_trips_filtrado_fim_semana = df_trips_filtrado[df_trips_filtrado.isweekend]

In [ ]:
df_trips_filtrado_dia_semana = df_trips_filtrado[~df_trips_filtrado.isweekend]

## Populares de Início - Dias de Semana

In [ ]:
popularidade_start_stations = df_trips_filtrado_dia_semana['Start Station ID'].value_counts().rename("Start Station Count")

In [ ]:
popularidade_start_stations.head()

In [ ]:
df_stations.head()

In [ ]:
df_start_station_count = df_stations.merge(popularidade_start_stations, right_index=True, left_index=True).sort_values("Start Station Count", ascending=False)

In [ ]:
df_start_station_count.head()

In [ ]:
fig = px.scatter_mapbox(
    df_start_station_count.head(10), lat="Station Latitude", lon="Station Longitude", size="Start Station Count",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()

### Conclusão Parcial

To be Done...

## Populares de Início - Fins de Semana

In [ ]:
popularidade_start_stations = df_trips_filtrado_fim_semana['Start Station ID'].value_counts().rename("Start Station Count")

In [ ]:
popularidade_start_stations.head()

In [ ]:
df_stations.head()

In [ ]:
df_start_station_count = df_stations.merge(popularidade_start_stations, right_index=True, left_index=True).sort_values("Start Station Count", ascending=False)

In [ ]:
df_start_station_count.head()

In [ ]:
fig = px.scatter_mapbox(
    df_start_station_count.head(10), lat="Station Latitude", lon="Station Longitude", size="Start Station Count",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()

### Conclusão Parcial

To be Done...

## Populares de Fim - Dias de Semana

In [ ]:
popularidade_end_stations = df_trips_filtrado_dia_semana['End Station ID'].value_counts().rename("End Station Count")

In [ ]:
popularidade_end_stations.head()

In [ ]:
df_stations.head()

In [ ]:
df_end_station_count = df_stations.merge(popularidade_end_stations, right_index=True, left_index=True).sort_values("End Station Count", ascending=False)

In [ ]:
df_end_station_count.head()

In [ ]:
fig = px.scatter_mapbox(
    df_end_station_count.head(10), lat="Station Latitude", lon="Station Longitude", size="End Station Count",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()

### Conclusão Parcial

To be Done...

## Populares de Fim - Fins de Semana

In [ ]:
popularidade_end_stations = df_trips_filtrado_fim_semana['End Station ID'].value_counts().rename("End Station Count")

In [ ]:
popularidade_end_stations.head()

In [ ]:
df_stations.head()

In [ ]:
df_end_station_count = df_stations.merge(popularidade_end_stations, right_index=True, left_index=True).sort_values("End Station Count", ascending=False)

In [ ]:
df_end_station_count.head()

In [ ]:
fig = px.scatter_mapbox(
    df_end_station_count.head(10), lat="Station Latitude", lon="Station Longitude", size="End Station Count",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()

### Conclusão Parcial

To be Done...

## Conclusões sobre as Popularidades

To be done...

# Análise dos Horários de Pico

In [ ]:
df_trips_filtrado_fim_semana = df_trips_filtrado[df_trips_filtrado.isweekend]

In [ ]:
df_trips_filtrado_dia_semana = df_trips_filtrado[~df_trips_filtrado.isweekend]

## Separado por Weekday

In [ ]:
df_horarios_pico = df_trips_filtrado.copy()

In [ ]:
df_horarios_pico['Hour'] = df_horarios_pico['Start Time'].dt.hour

In [ ]:
df_horarios_pico[['weekday', 'Hour']].head()

In [ ]:
hour_distribution = pd.pivot_table(df_horarios_pico[['weekday', 'Hour']], columns='weekday', index='Hour', aggfunc=len)

In [ ]:
px.line(hour_distribution)

### Conclusão

To be done...

# Análise do Uso por Dia da Semana

In [ ]:
df_trips_filtrado['weekday'].value_counts().sort_index().plot.bar()

# Análise de Sazonalidade

## Construção da Série Temporal

In [ ]:
df_time_series = df_trips_filtrado.copy()

In [ ]:
df_time_series['date-hour'] = pd.to_datetime(
    df_time_series['Start Time'].apply(
        lambda dt: dt.strftime("%Y-%m-%d %H")
    ),
    format="%Y-%m-%d %H"
)

In [ ]:
usage_series = df_time_series.groupby(['date-hour']).size()

In [ ]:
usage_series.head()

In [ ]:
# Precisamos que o dataframe tenha a propriedade de frequência por hora
usage_series = usage_series.asfreq('h')

In [ ]:
# Precisamos preencher os buracos vazios com uma interpolação
usage_series = usage_series.interpolate()

In [ ]:
usage_series.plot(figsize=(15,5))

## Análise da Decomposição de Sazonalidade

No estudo de séries temporais a sazonalidade é uma questão importante pois define o padrão de comportamento da série. Uma técnica de estudo de sazonalidade é a partir da decomposição em três partes:

* **Tendência:** Define a análise macro do comportamento da curva, que é a tendência geral para cada ponto da série.
* **Sazonalidade:** Define os padrões mais localizados/micros da curva.
* **Resíduo:** Todo o restante que não pode ser predito. Essa é uma medida importante porque em teoria deve ser aleatória, não demonstrando padrões nos resíduos. Se houverem razões podemos concluir que nosso modelo de decomposição não entende completamente os dados e está perdendo padrões.

https://machinelearningmastery.com/decompose-time-series-data-trend-seasonality/

https://otexts.com/fpp3/

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
result = seasonal_decompose(usage_series, model='additive')

In [ ]:
result.trend.plot(figsize=(15,5))

Podemos ver que a tendência capturou uma alta de acordo com os meses mais centrais do ano. Além disso, podemos ver que ela capturou curvas semanais, indicando o comportamento de sazonalidades semanais.

**Importante**: A tendência deve ser uma curva muito linear, pois não deve levar em consideração as volatilidades micro. O significado dessa sazonalidade embutida na tendência é que nossa componente de sazonalidade não deu conta de descobrir todos os padrões. Isso porque podemos encontrar três sazonalidades nos nossos dados: horária, diária e semanal. Podemos até encontrar uma anual se quisermos.

In [ ]:
result.trend[:1000].plot(figsize=(15,5))

Na análise da sazonalidade podemos ver que capturou a sazonalidade diária. Em uma análise mais profunda podemos ainda ver que essa é a sazonalidade dos dias de semana, visto que o comportamento dos fins de semana é diferente. Isso mostra como as múltiplas sazonalidades não foram captadas.

In [ ]:
result.seasonal[:100].plot(figsize=(15,5))

Com os resíduos também podemos ver as sazonalidades que escaparam do modelo.

In [ ]:
result.resid[:1000].plot(figsize=(15,5))

In [ ]:
result.observed[:1000].plot(figsize=(15,5))